In [1]:
import pandas as pd
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

credit = pd.read_csv(PATH + 'credit_card_balance.csv')
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')

'done'

'done'

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 0.1) # change const to 0.1

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
# ignore = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT',
#           'AMT_INST_MIN_REGULARITY', 'AMT_RECEIVABLE_PRINCIPAL']
# credit = credit[[x for x in credit.columns if x not in ignore]]
credit.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [4]:
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()
# For each credit_card(prev):
# 1. count num of records;
# 2. count num and ratio of status: active approved
# 3. Add shift

#1. count num of records in a prev id
x = credit[['SK_ID_PREV', 'SK_ID_CURR','MONTHS_BALANCE']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'MONTHS_BALANCE':'prev_count'})
# count agg fun for num of records in a prev id
tmp = numerical(x, 'SK_ID_CURR','prev_count',['min','max','std','mean'])
df = df.merge(tmp, on  = 'SK_ID_CURR', how = 'left')

del tmp,x
gc.collect()

_df = credit[['NAME_CONTRACT_STATUS']]
m = pd.get_dummies(_df['NAME_CONTRACT_STATUS'])
credit = pd.concat([credit, m], axis = 1)

In [5]:
credit.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'Active', 'Approved', 'Completed', 'Demand', 'Refused',
       'Sent proposal', 'Signed'],
      dtype='object')

In [6]:
# for each prev id, count how many complete, active are in there
x = credit[['SK_ID_PREV', 'SK_ID_CURR','Active']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'Active':'prev_active'})
y = credit[['SK_ID_PREV', 'SK_ID_CURR','Completed']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'Completed':'prev_complete'})
tmp1 = numerical(x, 'SK_ID_CURR','prev_active',['min','max','std','mean'])
tmp2 = numerical(y, 'SK_ID_CURR','prev_complete',['min','max','std','mean'])
df = df.merge(tmp1, on  = 'SK_ID_CURR', how = 'left')
df = df.merge(tmp2, on  = 'SK_ID_CURR', how = 'left')

del x,y,tmp1,tmp2
gc.collect()

161

In [7]:
print(df.shape)

(356255, 13)


In [8]:


merged_col = 'SK_ID_CURR'

# minus
name1 = minus_name('SK_DPD', 'SK_DPD_DEF')
credit[name1] = minus(credit, 'SK_DPD', 'SK_DPD_DEF')

# ratio
numerator = [x for x in credit.columns if 'AMT' in x] + ['SK_DPD', 'SK_DPD_DEF', name1]
denominator = [x for x in credit.columns if 'CNT' in x] + numerator + ['MONTHS_BALANCE']
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in credit.columns):
            credit[ratio_name(x, y)] = ratio(credit, x, y)
print('ratio\t', df.shape)
            
# one hot encoding sum
df = df.merge(one_hot(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
print('one hot sum\t', df.shape)

# count
df = df.merge(count(credit, merged_col, 'NAME_CONTRACT_STATUS'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print('count\t', df.shape)

# positive count
for x in ['SK_DPD', 'SK_DPD_DEF', name1]:
    df = df.merge(positive_count(credit, merged_col, x), on=merged_col, how='left')
print('positive count\t', df.shape)
    
# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)
print('categorical ratio\t', df.shape)
        
# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS', 'MONTHS_BALANCE']
for x in set(credit.columns) - set(ignored):
    df = df.merge(numerical(credit, merged_col, x, ['mean', 'median', 'sum', 'std']), on=merged_col, how='left')
print('numerical\t', df.shape)




ratio	 (356255, 13)
one hot sum	 (356255, 20)
count	 (356255, 21)
positive count	 (356255, 24)
categorical ratio	 (356255, 34)
numerical	 (356255, 922)


In [9]:

# # For each credit_card(prev):
# # 1. count num of records;
# # 2. count num and ratio of status: active approved
# # 3. Add shift

# #1. count num of records in a prev id
# x = credit[['SK_ID_PREV', 'SK_ID_CURR','MONTHS_BALANCE']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'MONTHS_BALANCE':'prev_count'})
# # count agg fun for num of records in a prev id
# tmp = numerical(x, 'SK_ID_CURR','prev_count',['min','max','std','mean'])
# df = df.merge(tmp, on  = 'SK_ID_CURR', how = 'left')

# del tmp,x
# gc.collect()

# _df = credit[['NAME_CONTRACT_STATUS']]
# m = pd.get_dummies(_df['NAME_CONTRACT_STATUS'])
# credit = pd.concat([credit, m], axis = 1)
# x = credit[['SK_ID_PREV', 'SK_ID_CURR','Active']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'Active':'prev_active'})
# y = credit[['SK_ID_PREV', 'SK_ID_CURR','Completed']].groupby(['SK_ID_PREV', 'SK_ID_CURR']).count().reset_index().rename(index = str, columns = {'Completed':'prev_complete'})
# tmp1 = numerical(x, 'SK_ID_CURR','prev_active',['min','max','std','mean'])
# tmp2 = numerical(y, 'SK_ID_CURR','prev_complete',['min','max','std','mean'])
# df = df.merge(tmp1, on  = 'SK_ID_CURR', how = 'left')
# df = df.merge(tmp2, on  = 'SK_ID_CURR', how = 'left')

In [10]:

columns = []
for x in df.columns:
    tmp = 'creditcard_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns
'done'

'done'

In [11]:
print(df.columns.values)
df.shape

['SK_ID_CURR' 'creditcard_prev_count_min' 'creditcard_prev_count_max'
 'creditcard_prev_count_std' 'creditcard_prev_count_mean'
 'creditcard_prev_active_min' 'creditcard_prev_active_max'
 'creditcard_prev_active_std' 'creditcard_prev_active_mean'
 'creditcard_prev_complete_min' 'creditcard_prev_complete_max'
 'creditcard_prev_complete_std' 'creditcard_prev_complete_mean'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Active'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Approved'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Completed'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Demand'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Refused'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Sent proposal'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Signed'
 'creditcard_count_SK_ID_CURR'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD_DEF'
 'creditcard_positivecount_SK_ID_CURR_SK_DPD_minus_SK_DPD_DEF'
 'creditcard_onehot_NAME_CONTRACT_STATUS_Active_divide_prev_count_min'
 'creditc

(356255, 922)

In [12]:
df.to_pickle(PATH + 'test/credit2curr_2.pkl')
'done'

'done'

In [13]:
print(train.shape, test.shape)

(307511, 122) (48744, 121)
